#Loading important dependencies

In [ ]:
!pip install PyPDF2

In [ ]:
import numpy as np
import PyPDF2
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfVectorizer
import re 

In [ ]:
pdf_file = open('/content/DeepSentiPeer.pdf','rb')

In [ ]:
pdf_read = PyPDF2.PdfFileReader(pdf_file)

In [ ]:
pdf_read.numPages

11

In [ ]:
page = pdf_read.getPage(0)

In [ ]:
page.extractText()

'DeepSentiPeer\n:HarnessingSentimentinReviewTextsToRecommend\nPeerReviewDecisions\nTirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya\nDepartmentofComputerScienceandEngineering\nIndianInstituteofTechnologyPatna,India\n(tirthankar.pcs16,rajeev.ee15,asif,pb)@iitp.ac.in\nAbstract\nAutomaticallyvalidatingaresearchartefactis\noneofthefrontiersinIntelligence\n(AI)thatdirectlybringsitclosetocompeting\nwithhumanintellectandintuition.Although\ncriticizedsometimes,theexistingpeerreview\nsystemstillstandsasthebenchmarkofre-\nsearchvalidation.Thepresent-daypeerreview\nprocessisnotstraightforwardanddemands\nprofounddomainknowledge,expertise,and\nintelligenceofhumanreviewer(s),whichis\nsomewhatelusivewiththecurrentstateofAI.\nHowever,thepeerreviewtexts,whichcontains\nrichsentimentinformationofthereviewer,re-\nhis/heroverallattitudetowardsthere-\nsearchinthepaper,couldbeavaluableen-\ntitytopredicttheacceptanceorrejectionof\nthemanuscriptunderconsideration.Herein\nthiswork,weinvestigatetherol

In [ ]:
output = ""
for i in range(pdf_read.numPages):
    page = pdf_read.getPage(i)
    output += page.extractText()

In [ ]:
output

"DeepSentiPeer\n:HarnessingSentimentinReviewTextsToRecommend\nPeerReviewDecisions\nTirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya\nDepartmentofComputerScienceandEngineering\nIndianInstituteofTechnologyPatna,India\n(tirthankar.pcs16,rajeev.ee15,asif,pb)@iitp.ac.in\nAbstract\nAutomaticallyvalidatingaresearchartefactis\noneofthefrontiersinIntelligence\n(AI)thatdirectlybringsitclosetocompeting\nwithhumanintellectandintuition.Although\ncriticizedsometimes,theexistingpeerreview\nsystemstillstandsasthebenchmarkofre-\nsearchvalidation.Thepresent-daypeerreview\nprocessisnotstraightforwardanddemands\nprofounddomainknowledge,expertise,and\nintelligenceofhumanreviewer(s),whichis\nsomewhatelusivewiththecurrentstateofAI.\nHowever,thepeerreviewtexts,whichcontains\nrichsentimentinformationofthereviewer,re-\nhis/heroverallattitudetowardsthere-\nsearchinthepaper,couldbeavaluableen-\ntitytopredicttheacceptanceorrejectionof\nthemanuscriptunderconsideration.Herein\nthiswork,weinvestigatetherol

# Preprocessing the data

In [ ]:
text = re.sub(r'\[[0-9]*\]',' ',output)
text = re.sub(r'\s+',' ',text)
text = re.sub(r'\d',' ',text)
text = re.sub(r'\s+',' ',text)
text

"DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition.Although criticizedsometimes,theexistingpeerreview systemstillstandsasthebenchmarkofre- searchvalidation.Thepresent-daypeerreview processisnotstraightforwardanddemands profounddomainknowledge,expertise,and intelligenceofhumanreviewer(s),whichis somewhatelusivewiththecurrentstateofAI. However,thepeerreviewtexts,whichcontains richsentimentinformationofthereviewer,re- his/heroverallattitudetowardsthere- searchinthepaper,couldbeavaluableen- titytopredicttheacceptanceorrejectionof themanuscriptunderconsideration.Herein thiswork,weinvestigatetheroleofreviewers sentimentsembe

#Summarization using Text Rank Algorithm

In [ ]:
#mock data ->Introduction from DeepSentiPeer paper
text_data= """The rapid increase in research article submissions across different venues is posing a significant management challenge for the journal editors and conference program chairs.Among the load of works like assigning reviewers, ensuring timely receipt of reviews, slot-filling against the non-responding reviewer, taking informed decisions, communicating to the authors, etc., editors/program chairs are usually overwhelmed with many such demanding yet crucial tasks. However, the major hurdle lies in to decide the acceptance and rejection of the manuscripts based on the reviews received from the reviewers.The quality, randomness, bias, inconsistencies in peer reviews is well-debated across the academic community (Bornmann and Daniel, 2010). Due to the rise in article submissions and nonavailability of expert reviewers, editors/program chairs are sometimes left with no other options than to assign papers to the novice, out of domain reviewers which sometimes results in more inconsistencies and poor quality reviews. To study the arbitrariness inherent in the existing peer review system, organisers of the NIPS 2014 conference assigned 10% submissions to two different sets of reviewers and observed that the two committees disagreed for more than quarter of the papers (Langford and Guzdial, 2015). Again it is quite common that a paper rejected in one venue gets the cut in another with little or almost no improvement in quality. Many are of the opinion that the existing peer review system is fragile as it only depends on the view of a selected few (Smith, 2006). Moreover, even a preliminary study into the inners of the peer review system is itself very difficult because of data confidentiality and copyright issues of the publishers. However, the silver lining is that the peer review system is evolving with the likes of OpenReviews2 , author response periods/rebuttals, increased effective communications between authors and reviewers, open access initiatives, peer review workshops, review forms"""

In [ ]:
#loading spacy model
nlp_model = English()

In [ ]:
nlp_model.add_pipe(nlp_model.create_pipe('sentencizer'))

In [ ]:
#creating spacy document
document = nlp_model(text_data.replace("\n"," "))

In [ ]:
sentences = [sent.string.strip() for sent in document.sents]

In [ ]:
sentences

['The rapid increase in research article submis\x02sions across different venues is posing a signif\x02icant management challenge for the journal ed\x02itors and conference program chairs.',
 'Among the load of works like assigning reviewers, ensur\x02ing timely receipt of reviews, slot-filling against the non-responding reviewer, taking informed de\x02cisions, communicating to the authors, etc.,',
 'edi\x02tors/program chairs are usually overwhelmed with many such demanding yet crucial tasks.',
 'However, the major hurdle lies in to decide the acceptance and rejection of the manuscripts based on the re\x02views received from the reviewers.',
 'The quality, randomness, bias, inconsistencies in peer reviews is well-debated across the aca\x02demic community (Bornmann and Daniel, 2010).',
 'Due to the rise in article submissions and non\x02availability of expert reviewers, editors/program chairs are sometimes left with no other options than to assign papers to the novice, out of domain re

In [ ]:
organized_sentences = {k:v for v,k in enumerate(sentences)}
organized_sentences

{'Again it is quite common that a paper rejected in one venue gets the cut in another with little or almost no improve\x02ment in quality.': 7,
 'Among the load of works like assigning reviewers, ensur\x02ing timely receipt of reviews, slot-filling against the non-responding reviewer, taking informed de\x02cisions, communicating to the authors, etc.,': 1,
 'Due to the rise in article submissions and non\x02availability of expert reviewers, editors/program chairs are sometimes left with no other options than to assign papers to the novice, out of domain reviewers which sometimes results in more incon\x02sistencies and poor quality reviews.': 5,
 'However, the major hurdle lies in to decide the acceptance and rejection of the manuscripts based on the re\x02views received from the reviewers.': 3,
 'However, the silver lin\x02ing is that the peer review system is evolving with the likes of OpenReviews2 , author response peri\x02ods/rebuttals, increased effective communications between auth

In [ ]:
#TFIDF feature extraction
cv = TfidfVectorizer()
X = cv.fit_transform(sentences) #to transform sentences into tfidf vectors
X

<11x196 sparse matrix of type '<class 'numpy.float64'>'
	with 281 stored elements in Compressed Sparse Row format>

In [ ]:
#sentence scores 
scores = np.array(X.sum(axis=1)).ravel()
scores

array([4.92026898, 5.15740477, 3.70879202, 4.25052158, 4.2596318 ,
       5.86759305, 5.72986023, 4.67578207, 4.72122314, 4.75828941,
       5.55627753])

In [ ]:
len(sentences)

11

In [ ]:
len(scores)

11

In [ ]:
#getting top n sequences
N=7
top_n = [sentences[ind] for ind in np.argsort(scores, axis=0)[::-1][:N]]
top_n

['Due to the rise in article submissions and non\x02availability of expert reviewers, editors/program chairs are sometimes left with no other options than to assign papers to the novice, out of domain reviewers which sometimes results in more incon\x02sistencies and poor quality reviews.',
 'To study the arbitrariness inherent in the existing peer review system, organisers of the NIPS 2014 conference assigned 10% submissions to two different sets of reviewers and observed that the two commit\x02tees disagreed for more than quarter of the papers (Langford and Guzdial, 2015).',
 'However, the silver lin\x02ing is that the peer review system is evolving with the likes of OpenReviews2 , author response peri\x02ods/rebuttals, increased effective communications between authors and reviewers, open access initia\x02tives, peer review workshops, review forms',
 'Among the load of works like assigning reviewers, ensur\x02ing timely receipt of reviews, slot-filling against the non-responding revi

In [ ]:
#sentence ordering wrt organized sentences
mapped_top_n_sent = [(sentence,organized_sentences[sentence]) for sentence in top_n]
mapped_top_n_sent

[('DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition.',
  0),
 ('Adataset ofpeerreviews(peerread):Collection,insightsand NLPapplications .In Proceedingsofthe Con- ferenceoftheNorthAmericanChapteroftheAsso- ciationforComputationalLinguistics:HumanLan- guageTechnologies,NAACL-HLT ,NewOr- leans,Louisiana,USA,June - , ,Volume (LongPapers) ,pages Œ .',
  270),
 (".However,fromtheheatmapinFigure wecanseethatthereviewer'ssentiments(com- pound/positive)embeddedwithinthereviewtexts havevisiblecorrelationswiththeaspectslike Rec- ommendation , Appropriateness and OverallDeci- sion .Thisalsosecondsourrecentthatde- terminingthescop

In [ ]:
for i in mapped_top_n_sent:
  print(i)

('DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition.', 0)
('Adataset ofpeerreviews(peerread):Collection,insightsand NLPapplications .In Proceedingsofthe Con- ferenceoftheNorthAmericanChapteroftheAsso- ciationforComputationalLinguistics:HumanLan- guageTechnologies,NAACL-HLT ,NewOr- leans,Louisiana,USA,June - , ,Volume (LongPapers) ,pages Œ .', 270)
(".However,fromtheheatmapinFigure wecanseethatthereviewer'ssentiments(com- pound/positive)embeddedwithinthereviewtexts havevisiblecorrelationswiththeaspectslike Rec- ommendation , Appropriateness and OverallDeci- sion .Thisalsosecondsourrecentthatde- terminingthescopeorapprop

In [ ]:
#ordering top n sentences in original order
mapped_top_n_sent = sorted(mapped_top_n_sent, key = lambda x: x[1])
mapped_top_n_sent

[('DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition.',
  0),
 (".However,fromtheheatmapinFigure wecanseethatthereviewer'ssentiments(com- pound/positive)embeddedwithinthereviewtexts havevisiblecorrelationswiththeaspectslike Rec- ommendation , Appropriateness and OverallDeci- sion .Thisalsosecondsourrecentthatde- terminingthescopeorappropriatenessofanarti- cletoavenueistheessentialstepinpeerre- view( Ghosaletal. ,",
  47),
 ('Adataset ofpeerreviews(peerread):Collection,insightsand NLPapplications .In Proceedingsofthe Con- ferenceoftheNorthAmericanChapteroftheAsso- ciationforComputationalLinguistics:HumanLan- guageTechno

In [ ]:
ordered_sentences = [i[0] for i in mapped_top_n_sent]
ordered_sentences

['DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition.',
 ".However,fromtheheatmapinFigure wecanseethatthereviewer'ssentiments(com- pound/positive)embeddedwithinthereviewtexts havevisiblecorrelationswiththeaspectslike Rec- ommendation , Appropriateness and OverallDeci- sion .Thisalsosecondsourrecentthatde- terminingthescopeorappropriatenessofanarti- cletoavenueistheessentialstepinpeerre- view( Ghosaletal. ,",
 'Adataset ofpeerreviews(peerread):Collection,insightsand NLPapplications .In Proceedingsofthe Con- ferenceoftheNorthAmericanChapteroftheAsso- ciationforComputationalLinguistics:HumanLan- guageTechnologies,NAACL-HLT

In [ ]:
abstract = " ".join(ordered_sentences)
abstract

"DeepSentiPeer :HarnessingSentimentinReviewTextsToRecommend PeerReviewDecisions TirthankarGhosal,RajeevVerma,AsifEkbal,PushpakBhattacharyya DepartmentofComputerScienceandEngineering IndianInstituteofTechnologyPatna,India (tirthankar.pcs ,rajeev.ee ,asif,pb)@iitp.ac.in Abstract Automaticallyvalidatingaresearchartefactis oneofthefrontiersinIntelligence (AI)thatdirectlybringsitclosetocompeting withhumanintellectandintuition. .However,fromtheheatmapinFigure wecanseethatthereviewer'ssentiments(com- pound/positive)embeddedwithinthereviewtexts havevisiblecorrelationswiththeaspectslike Rec- ommendation , Appropriateness and OverallDeci- sion .Thisalsosecondsourrecentthatde- terminingthescopeorappropriatenessofanarti- cletoavenueistheessentialstepinpeerre- view( Ghosaletal. , Adataset ofpeerreviews(peerread):Collection,insightsand NLPapplications .In Proceedingsofthe Con- ferenceoftheNorthAmericanChapteroftheAsso- ciationforComputationalLinguistics:HumanLan- guageTechnologies,NAACL-HLT ,NewOr- 

#Using transformers

#Loading important dependencies

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

#loading summarizing pipeline 

In [ ]:
summarization = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [ ]:
text= """The rapid increase in research article submissions across different venues is posing a significant management challenge for the journal editors and conference program chairs.Among the load of works like assigning reviewers, ensuring timely receipt of reviews, slot-filling against the non-responding reviewer, taking informed decisions, communicating to the authors, etc., editors/program chairs are usually overwhelmed with many such demanding yet crucial tasks. However, the major hurdle lies in to decide the acceptance and rejection of the manuscripts based on the reviews received from the reviewers.The quality, randomness, bias, inconsistencies in peer reviews is well-debated across the academic community (Bornmann and Daniel, 2010). Due to the rise in article submissions and nonavailability of expert reviewers, editors/program chairs are sometimes left with no other options than to assign papers to the novice, out of domain reviewers which sometimes results in more inconsistencies and poor quality reviews. To study the arbitrariness inherent in the existing peer review system, organisers of the NIPS 2014 conference assigned 10% submissions to two different sets of reviewers and observed that the two committees disagreed for more than quarter of the papers (Langford and Guzdial, 2015). Again it is quite common that a paper rejected in one venue gets the cut in another with little or almost no improvement in quality. Many are of the opinion that the existing peer review system is fragile as it only depends on the view of a selected few (Smith, 2006). Moreover, even a preliminary study into the inners of the peer review system is itself very difficult because of data confidentiality and copyright issues of the publishers. However, the silver lining is that the peer review system is evolving with the likes of OpenReviews2 , author response periods/rebuttals, increased effective communications between authors and reviewers, open access initiatives, peer review workshops, review forms"""

#Abstractive Summarization

In [ ]:
summarization(text, max_length=200, min_length=50, do_sample=False)

[{'summary_text': ' The rapid increase in research article submissions across different venues is posing a management challenge for journal edicatingitors and conference program chairs . The quality, randomness, bias, inconsistencies in peer reviews is well-debated across the aca.demic community (Bornmann and Daniel, 2010)'}]